# Backtesting with backtrader library
Notes:  
Timestamp must be in the exact format specified for datafeed   **Timestamp needs to be first column index**  
Params for datafeed need to be specified as tuple of tuples  
Within strategy data is accessed via self.datas[0] for ex: self.datas[0].high_delta[0] would get you the first timeperiod's high_delta  
The current data has already happened and cannot be used to execute an order (ex. you cannot look at the current close price and simultaneously buy the current close, the order will be executed at open instead), orders will be executed on the following day.

In [1]:
import backtrader as bt
import datetime
import pandas as pd

## Define Strategies and Data utils

In [2]:
class BasicStrategy(bt.Strategy):
    def __init__(self):
        # To keep track of pending orders
        self.order = None
        print('***The limit sells for this strategy are valid for 1 day***')
        
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        elif order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)
            
            self.bar_executed = len(self)
        
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
        
        # set no pending order
        self.order = None


    def next(self):
        # STRATEGY 
        data = self.datas[0]
        self.log(f'Current Portfolio Value : {self.broker.get_value()}')
        
        # cancel if there is an order pending, this strategy should have 1 working order per day
        if self.order:
            self.log('ORDER CANCELLED')
            self.cancel(self.order)
        
        # Check if we are in the market
        if not self.position:
            # BUY
            try:
                self.size = int(self.broker.get_cash() / self.datas[0].open[1])
            except:
                print('Size Exception. If at the end of data, ignore.')
            # invest if prediction looks good
            self.log(f'MARKET BUY CREATE {self.size} shares at next open, current close price: {data.close[0]}')
            self.buy(size=self.size) # market order buys at next open                      
                
        else:
            # place sell order at predicted high if predicted high is greater than current close price
            # TODO: Make prediction and close a filter in the class constructor (more optimal)
            if data.prediction[0] >= data.close[0]:
                self.log(f'LIMIT SELL CREATE {self.size} shares at {data.prediction[0]}')  
                self.sell(exectype=bt.Order.Limit,
                             price=data.prediction[0],
                             valid=data.datetime.date(0) + datetime.timedelta(days=2),
                             size=self.size)
        # if prediction is less than current value sell at open (ASAP)
            else:
                self.log('MARKET SELL CREATE. PREDICTION < CURRENT CLOSE')
                self.sell(size=self.size)
        

In [3]:
class BuyAndHold(bt.Strategy):
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

      
    def start(self):
        self.val_start = self.broker.get_cash()  # keep the starting cash

    def nextstart(self):
        # Buy all the available cash
        size = int(self.broker.get_cash() / self.datas[0].open[1])
        self.buy(size=size)

    def stop(self):
        # calculate the actual returns
        self.roi = (self.broker.get_value() / self.val_start) - 1.0
        print(f'Stop price: {self.datas[0].close[0]}')
        print('ROI:        {:.2f}%'.format(100.0 * self.roi))

In [4]:
def prepare_data(data, fromdate, todate, filepath):
    """Prepare data for backtrader datafeed object
        Returns prepared data filepath and params for the GenericCSVData class, also returns columns used"""
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    # start param setup for backtrader
    start = data['timestamp'].iloc[0]
    end = data['timestamp'].iloc[-1]
    from_to = [(start.year, start.month, start.day), (end.year, end.month, end.day)]
    # Backtrader string format
    data['timestamp'] = data['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
    # backtrader data feed class needs a file path, this could be a temp file that's constantly overwritten
    data.to_csv(filepath)
    
    starting_params = [
    ('fromdate', datetime.datetime(*from_to[0])),
    ('todate', datetime.datetime(*from_to[1])),
    ('nullvalue', 0.0),
    ('dtformat', ('%Y-%m-%d %H:%M:%S')),
    ('tmformat', ('%H:%M:%S')),
    ('datetime', 1)]
    # skip nonfeatures (timestamp)
    cols = data.columns[1:]
    # get column position for each indicator and add to starting params list
    i = 2 # starting index since others are reserved
    for indicator in cols:
        starting_params.append((indicator, i))
        i+=1
    final_params = tuple(starting_params)

    return filepath, final_params, cols 


## Import Data and Run Backtest

In [5]:
import os

In [6]:
def capstone_data_prep(pred_folder_name, pred_col_name, ticker_df_filepath, starting_cash=100000):
    """Main loop, preps data and executes backtrader on all stocks in the prediction folder"""
    ticker_backtesting_dict = {}
    for pred_file in os.listdir(f'../data/ticker_predictions/{pred_folder_name}'):
        # collect data
        ticker_name = pred_file.split('_')[0]
        pred_df = pd.read_csv(f'../data/ticker_predictions/{pred_folder_name}/{pred_file}')
        ticker_df = pd.read_csv(ticker_df_filepath + f'/{ticker_name}_full_data.csv')
        
        # make timestamp in column position 0, SPECIFIC TO FULL DATA IN FILE 
        ticker_df = ticker_df.rename({'reportperiod':'timestamp'}, axis=1)
        ticker_cols = list(ticker_df.columns)
        ticker_cols[5] = 'ts_cpy'
        ticker_cols[0] = 'timestamp'
        ticker_df.columns = ticker_cols
        ticker_df['timestamp'] = ticker_df['ts_cpy']
        # merge prediction data with full data, on key = timestamp
        ticker_df = ticker_df.merge(pred_df.loc[:, ['timestamp', pred_col_name]], on='timestamp')
        # new prediction column name = "prediction", important because backtester strategy looks for this column name
        ticker_df = ticker_df.rename({pred_col_name:'prediction'}, axis=1)
        # remove all non-prediction rows in prediction field
        ticker_df = ticker_df[ticker_df.prediction > 0]
        # skip nonfeatures, keep timestamp, SPECIFIC TO FULL DATA IN FILE 
        nonfeatures = ticker_df.columns[1:7]
        features = [col for col in ticker_df.columns if col not in nonfeatures]
        # get index of start and end dates for trading
        ticker_df = ticker_df[features]
        ticker_df.columns = [c.lower() for c in ticker_df.columns]
        idx1 = ticker_df.prediction[ticker_df.prediction > 0].index[0]
        idx2 = ticker_df.prediction[ticker_df.prediction > 0].index[-1]
        start_date = pred_df.timestamp[idx1]
        end_date  = pred_df.timestamp[idx2]
        
        # Prepare data for backtrader
        prep_data = prepare_data(ticker_df.copy(), start_date, end_date, 'temp/prep_data.csv')
        class DataFeed(bt.feeds.GenericCSVData):
            lines = tuple(prep_data[2])
            params = prep_data[1]
        
        print(f"--------Ticker Name-------: {ticker_name}")
        ##### BUY AND HOLD #######
        print('BUY AND HOLD')
        # initialize everything and run strategy
        cerebro = bt.Cerebro(cheat_on_open=True)
        cerebro.broker.setcash(starting_cash)
        #cerebro.addsizer(bt.sizers.FixedSize, stake=10)
        print(f'Starting Portfolio Value: {cerebro.broker.getvalue()}')
        data = DataFeed(dataname=prep_data[0])
        cerebro.adddata(data)
        cerebro.addstrategy(BuyAndHold)
        cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Months, _name = 'sharpe')
        cerebro.addanalyzer(bt.analyzers.DrawDown, _name = 'dd')
        cerebro.addanalyzer(bt.analyzers.Returns, _name = 'returns')
        buy_hold = cerebro.run()
        buy_hold_ret = cerebro.broker.getvalue() / starting_cash - 1
        print(f'Final Portfolio Value: {cerebro.broker.getvalue()}\n')
        print('STRATEGY INFO:')
        print(buy_hold[0].analyzers.sharpe.get_analysis())
        print(buy_hold[0].analyzers.dd.get_analysis())
        # rtot is total log returns over the strategy time period
        print(buy_hold[0].analyzers.returns.get_analysis())

        #### MAIN STRATEGY #####
        cerebro = bt.Cerebro(cheat_on_open=True)
        cerebro.broker.setcash(starting_cash)
        print(f'Starting Portfolio Value: {cerebro.broker.getvalue()}')
        data = DataFeed(dataname=prep_data[0])
        cerebro.adddata(data)
        cerebro.addstrategy(BasicStrategy)
        cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Months, _name = 'sharpe')
        cerebro.addanalyzer(bt.analyzers.DrawDown, _name = 'dd')
        cerebro.addanalyzer(bt.analyzers.Returns, _name = 'returns')
        back = cerebro.run()
        strat_ret = cerebro.broker.getvalue() / starting_cash - 1
        print(f'Final Portfolio Value: {cerebro.broker.getvalue()}\n')
        print('STRATEGY INFO:')
        print(back[0].analyzers.sharpe.get_analysis())
        print(back[0].analyzers.dd.get_analysis())
        # rtot is total log returns over the strategy time period
        print(back[0].analyzers.returns.get_analysis())
        print(f"For {ticker_name}, return was {buy_hold_ret*100}% for buy and hold vs {strat_ret*100}%  for strategy")
        print('\n'*5)

        ticker_backtesting_dict[ticker_name] = cerebro
        
    return ticker_backtesting_dict

In [7]:
# pred_folder_name, pred_col_name, ticker_df_filepath, starting_cash=100000

cerebro_dict = capstone_data_prep('rfreg_preds', 'prediction', '../data/ticker_data')
cerebro_dict

--------Ticker Name-------: ADSK
BUY AND HOLD
Starting Portfolio Value: 100000
2019-09-19, BUY EXECUTED, 151.39
Stop price: 329.07000732421875
ROI:        117.27%
Final Portfolio Value: 217268.8052368164

STRATEGY INFO:
OrderedDict([('sharperatio', 0.35206001119841246)])
AutoOrderedDict([('len', 52), ('drawdown', 3.8551895466552977), ('moneydown', 8711.987915039062), ('max', AutoOrderedDict([('len', 136), ('drawdown', 35.45057378420961), ('moneydown', 49341.59637451175)]))])
OrderedDict([('rtot', 0.7759651347648641), ('ravg', 0.0014343163304341294), ('rnorm', 0.4354059702078554), ('rnorm100', 43.54059702078554)])
Starting Portfolio Value: 100000
***The limit sells for this strategy are valid for 1 day***
2019-09-18, Current Portfolio Value : 100000.0
2019-09-18, MARKET BUY CREATE 660 shares at next open, current close price: 151.11000061035156
2019-09-19, BUY EXECUTED, 151.39
2019-09-19, Current Portfolio Value : 101313.40362548828
2019-09-19, LIMIT SELL CREATE 660 shares at 155.377909

2020-08-28, SELL EXECUTED, 246.03
2020-08-28, Current Portfolio Value : 144822.2531326776
2020-08-28, MARKET BUY CREATE 587 shares at next open, current close price: 247.38999938964844
2020-08-31, BUY EXECUTED, 246.33
2020-08-31, Current Portfolio Value : 144452.44026646667
2020-08-31, LIMIT SELL CREATE 587 shares at 248.64924408427586
2020-09-01, SELL EXECUTED, 248.65
2020-09-01, Current Portfolio Value : 146183.64833531843
2020-09-01, MARKET BUY CREATE 572 shares at next open, current close price: 253.50999450683597
2020-09-02, BUY EXECUTED, 255.48
2020-09-02, Current Portfolio Value : 149541.29427037702
2020-09-02, LIMIT SELL CREATE 572 shares at 264.3587205380235
2020-09-03, Current Portfolio Value : 138524.56868443955
2020-09-03, LIMIT SELL CREATE 572 shares at 245.22470139577047
2020-09-04, Current Portfolio Value : 133828.45357213487
2020-09-04, LIMIT SELL CREATE 572 shares at 237.23088530025103
2020-09-08, Current Portfolio Value : 128383.01112828721
2020-09-08, LIMIT SELL CREA

2021-07-02, SELL EXECUTED, 297.30
2021-07-02, Current Portfolio Value : 150344.0003867733
2021-07-02, MARKET BUY CREATE 504 shares at next open, current close price: 297.739990234375
2021-07-06, BUY EXECUTED, 298.19
2021-07-06, Current Portfolio Value : 149038.6422324764
2021-07-06, LIMIT SELL CREATE 504 shares at 299.2032535897995
2021-07-07, SELL EXECUTED, 299.20
2021-07-07, Current Portfolio Value : 150854.67896556348
2021-07-07, MARKET BUY CREATE 516 shares at next open, current close price: 296.1099853515625
2021-07-08, BUY EXECUTED, 292.30
2021-07-08, Current Portfolio Value : 151974.40589427442
2021-07-08, LIMIT SELL CREATE 516 shares at 298.082343272983
2021-07-09, Current Portfolio Value : 152443.96778392285
2021-07-09, LIMIT SELL CREATE 516 shares at 298.980570640987
2021-07-12, Current Portfolio Value : 150663.76148509473
2021-07-12, LIMIT SELL CREATE 516 shares at 295.5494220226241
2021-07-13, SELL EXECUTED, 295.55
2021-07-13, Current Portfolio Value : 152531.38702806566
20

--------Ticker Name-------: HD
BUY AND HOLD
Starting Portfolio Value: 100000
2019-09-19, BUY EXECUTED, 231.15
Stop price: 368.5899963378906
ROI:        59.37%
Final Portfolio Value: 159374.0810546875

STRATEGY INFO:
OrderedDict([('sharperatio', 0.266248586298018)])
AutoOrderedDict([('len', 8), ('drawdown', 1.0009381519418883), ('moneydown', 1611.36474609375), ('max', AutoOrderedDict([('len', 143), ('drawdown', 38.354331184162646), ('moneydown', 40983.84448242186)]))])
OrderedDict([('rtot', 0.4660839639766037), ('ravg', 0.0008615230387737592), ('rnorm', 0.2424730713188748), ('rnorm100', 24.24730713188748)])
Starting Portfolio Value: 100000
***The limit sells for this strategy are valid for 1 day***
2019-09-18, Current Portfolio Value : 100000.0
2019-09-18, MARKET BUY CREATE 432 shares at next open, current close price: 230.8300018310547
2019-09-19, BUY EXECUTED, 231.15
2019-09-19, Current Portfolio Value : 98691.04052734374
2019-09-19, LIMIT SELL CREATE 432 shares at 230.00485633306045


2020-08-27, BUY EXECUTED, 292.22
2020-08-27, Current Portfolio Value : 146050.02029210815
2020-08-27, LIMIT SELL CREATE 506 shares at 291.1143297729338
2020-08-28, Current Portfolio Value : 144865.9821451355
2020-08-28, LIMIT SELL CREATE 506 shares at 288.77595714919437
2020-08-31, Current Portfolio Value : 144233.4821451355
2020-08-31, LIMIT SELL CREATE 506 shares at 287.50200170072213
2020-09-01, Current Portfolio Value : 144688.8790567566
2020-09-01, LIMIT SELL CREATE 506 shares at 288.40976916844903
2020-09-02, Current Portfolio Value : 145326.44399816284
2020-09-02, LIMIT SELL CREATE 506 shares at 289.68066209196843
2020-09-03, Current Portfolio Value : 138966.02029210815
2020-09-03, LIMIT SELL CREATE 506 shares at 277.11846260956133
2020-09-04, Current Portfolio Value : 136451.1996744324
2020-09-04, LIMIT SELL CREATE 506 shares at 272.4292471216663
2020-09-08, Current Portfolio Value : 136248.80276281128
2020-09-08, LIMIT SELL CREATE 506 shares at 271.6885396266381
2020-09-09, SE

...

2021-06-15, SELL EXECUTED, 339.17
2021-06-15, Current Portfolio Value : 124980.99459799685
2021-06-15, MARKET BUY CREATE 365 shares at next open, current close price: 341.79998779296875
2021-06-16, BUY EXECUTED, 342.11
2021-06-16, Current Portfolio Value : 125331.40261801638
2021-06-16, LIMIT SELL CREATE 365 shares at 346.095075052722
2021-06-17, SELL EXECUTED, 346.10
2021-06-17, Current Portfolio Value : 126435.55233892007
2021-06-17, MARKET BUY CREATE 366 shares at next open, current close price: 346.55999755859375
2021-06-18, BUY EXECUTED, 345.04
2021-06-18, Current Portfolio Value : 125582.76608403726
2021-06-18, LIMIT SELL CREATE 366 shares at 345.88614903342994
2021-06-21, Current Portfolio Value : 125483.95010503335
2021-06-21, LIMIT SELL CREATE 366 shares at 345.47705989614906
2021-06-22, SELL EXECUTED, 345.48
2021-06-22, Current Portfolio Value : 126595.51313346921
2021-06-22, MARKET BUY CREATE 365 shares at next open, current close price: 345.6199951171875
2021-06-23, BUY EXE

--------Ticker Name-------: XOM
BUY AND HOLD
Starting Portfolio Value: 100000
2019-09-17, BUY EXECUTED, 73.82
Stop price: 65.72000122070311
ROI:        -10.97%
Final Portfolio Value: 89032.60206604002

STRATEGY INFO:
OrderedDict([('sharperatio', 0.007655329138823791)])
AutoOrderedDict([('len', 542), ('drawdown', 10.967397933959974), ('moneydown', 10967.397933959975), ('max', AutoOrderedDict([('len', 542), ('drawdown', 57.368978553771974), ('moneydown', 57368.97855377197)]))])
OrderedDict([('rtot', -0.11616756798427419), ('ravg', -0.0002139365892896394), ('rnorm', -0.05248453522926926), ('rnorm100', -5.248453522926926)])
Starting Portfolio Value: 100000
***The limit sells for this strategy are valid for 1 day***
2019-09-16, Current Portfolio Value : 100000.0
2019-09-16, MARKET BUY CREATE 1354 shares at next open, current close price: 73.7300033569336
2019-09-17, BUY EXECUTED, 73.82
2019-09-17, Current Portfolio Value : 99119.89793395996
2019-09-17, LIMIT SELL CREATE 1354 shares at 73.75

2020-08-13, Current Portfolio Value : 78204.56359515384
2020-08-13, LIMIT SELL CREATE 1818 shares at 43.6085225702419
2020-08-14, Current Portfolio Value : 78549.9880336304
2020-08-14, LIMIT SELL CREATE 1818 shares at 43.694426440057036
2020-08-17, Current Portfolio Value : 77531.90553698734
2020-08-17, LIMIT SELL CREATE 1818 shares at 43.12298602429193
2020-08-18, SELL EXECUTED, 43.12
2020-08-18, Current Portfolio Value : 78409.97523876921
2020-08-18, MARKET BUY CREATE 1851 shares at next open, current close price: 42.43000030517578
2020-08-19, BUY EXECUTED, 42.35
2020-08-19, Current Portfolio Value : 77688.08636852996
2020-08-19, LIMIT SELL CREATE 1851 shares at 42.43758826862659
2020-08-20, Current Portfolio Value : 76503.4474982907
2020-08-20, LIMIT SELL CREATE 1851 shares at 41.80643312861315
2020-08-21, Current Portfolio Value : 75929.63495632903
2020-08-21, LIMIT SELL CREATE 1851 shares at 41.488483637193845
2020-08-24, Current Portfolio Value : 78169.35032269257
2020-08-24, LIM

2021-06-21, Current Portfolio Value : 102590.99852712666
2021-06-21, LIMIT SELL CREATE 1639 shares at 63.12590904967917
2021-06-22, SELL EXECUTED, 63.13
2021-06-22, Current Portfolio Value : 103469.35320945928
2021-06-22, MARKET BUY CREATE 1615 shares at next open, current close price: 63.790000915527344
2021-06-23, BUY EXECUTED, 64.05
2021-06-23, Current Portfolio Value : 103808.50173088262
2021-06-23, LIMIT SELL CREATE 1615 shares at 64.80851042876046
2021-06-24, Current Portfolio Value : 104228.39285942263
2021-06-24, LIMIT SELL CREATE 1615 shares at 65.04912460270718
2021-06-25, Current Portfolio Value : 104454.50419517708
2021-06-25, LIMIT SELL CREATE 1615 shares at 65.16078856811727
2021-06-28, Current Portfolio Value : 101789.74557014654
2021-06-28, LIMIT SELL CREATE 1615 shares at 63.571409535951176
2021-06-29, SELL EXECUTED, 63.57
2021-06-29, Current Portfolio Value : 102696.42468143157
2021-06-29, MARKET BUY CREATE 1637 shares at next open, current close price: 62.61999893188

{'ADSK': <backtrader.cerebro.Cerebro at 0x219cca94518>,
 'HD': <backtrader.cerebro.Cerebro at 0x219ccff9b70>,
 'MSFT': <backtrader.cerebro.Cerebro at 0x219cd3daac8>,
 'UNH': <backtrader.cerebro.Cerebro at 0x219ccb15ac8>,
 'WAT': <backtrader.cerebro.Cerebro at 0x219cdb20a20>,
 'XOM': <backtrader.cerebro.Cerebro at 0x219cde61a58>}

### Library additionally has plotting
Not sure how to reformat this, it's pretty ugly

In [8]:
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams.update({'font.size': 12}) 
cerebro_dict['UNH'].plot()

<IPython.core.display.Javascript object>

[[<Figure size 720x576 with 4 Axes>]]

### Ways to Improve:

Increase strategy trading volume. There are certain periods where sell orders are not going through for multiple days in a row, meaning that the price is lower than anticipated by model for that period. For example, if the last sell order didn't go through, we can find a way to use that information to inform our next sell order and make the prediction lower or try a market sell. Maybe train a "bear" model, one that only knows pain and suffering of bear markets, and only produces pessimistic high price predictions.   

Smarter entry points. Currently, buy orders are being executed at market price at the open. Returns may increase if entry constraints are increased.  

Resizing orders. Orders for this strategy baseline are currently 100% in or 100% out of position. This has beneifits if you want less exposure to the long term movement of the stocks. Depending on the situation and strategy, we might want to reserve a cash position to expose ourselves less to short term movements.  